In [1]:
import warnings
from itertools import combinations

import matplotlib
import openpyxl
from openpyxl import load_workbook
import pandas as pd
# import selector as selector
# from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
# import waterfall_chart
from matplotlib import pyplot
from matplotlib.ticker import MultipleLocator
# from ruamel.yaml import timestamp
from sklearn.feature_selection import SelectKBest, chi2, RFECV, SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_curve, auc, precision_score, make_scorer, log_loss, roc_auc_score
from pandas import read_csv
from pandas import set_option
from sklearn import metrics, datasets, clone
import numpy as np
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, KBinsDiscretizer
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit, validation_curve, \
    learning_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression, LassoCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier as GBDT
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.utils import shuffle
import seaborn as sns

warnings.filterwarnings('ignore')


In [2]:
########################################  import original radiomics data  ################################################
NHT_filepath = r'E:\Pystudy\Thyroid_P_N\data\results\data_use\mix\imputer\NHT_radiomics.xlsx'
PTMC_filepath = r'E:\Pystudy\Thyroid_P_N\data\results\data_use\mix\imputer\PTMC_radiomics.xlsx'
writer_1 = pd.ExcelWriter(NHT_filepath, engine='openpyxl', mode='a')
writer_2 = pd.ExcelWriter(PTMC_filepath, engine='openpyxl', mode='a')
df_1 = pd.read_excel(NHT_filepath)
df_2 = pd.read_excel(PTMC_filepath)

# NHT：df_1 to DataFrame
df_1_colNames = df_1.columns
df_1 = df_1.astype(np.float64)
df_1 = pd.DataFrame(df_1)
df_1.columns = df_1_colNames
print('df_1:', "\n",df_1.shape)

# PTMC：df_2 to DataFrame
df_2_colNames = df_2.columns
df_2 = df_2.astype(np.float64)
df_2 = pd.DataFrame(df_2)
df_2.columns = df_2_colNames
print('df_2:', "\n",df_2.shape)


df_1: 
 (114, 6720)
df_2: 
 (76, 6720)


In [13]:
######################################  NHT null check and statistics  ##################################################
# check all null
# df_1_Nan=df_1.isnull().sum()
# df_2_Nan=df_2.isnull().sum()
# print('df_1_NHT缺失值：',"\n", df_1_Nan,"\n")
# print('df_2_PTMC缺失值：',"\n", df_2_Nan,"\n")

# display columns of null and show total null number
NHT_missing_data = df_1.isnull().sum()
NHT_missing_data = NHT_missing_data[NHT_missing_data > 0].sort_values(ascending=False)
total_missing_values = NHT_missing_data.sum()
print("NHT Columns with missing values:")
print(NHT_missing_data)
print(f"Total missing values in df_1: {total_missing_values}")

# show number of NHT DataFrame and Percentage of missing values in NHT
total_data_points = df_1.size
print("NHT组学数据整体数量：", total_data_points) 
missing_percentage = (total_missing_values / total_data_points) * 100
print(f"Percentage of missing values in NHT: {missing_percentage:.2f}%")

NHT Columns with missing values:
T1_C_wavelet-HLH_firstorder_RobustMeanAbsoluteDeviation            8
T1_C_log-sigma-3-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation    8
T1_C_wavelet-LLL_firstorder_RobustMeanAbsoluteDeviation            8
T1_C_log-sigma-5-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation    8
T1_C_wavelet-LLH_firstorder_RobustMeanAbsoluteDeviation            8
                                                                  ..
T2_wavelet-LLH_glcm_DifferenceAverage                              1
T2_wavelet-LLH_glcm_DifferenceEntropy                              1
T2_wavelet-LLH_glcm_DifferenceVariance                             1
T2_wavelet-LLH_glcm_JointEnergy                                    1
T2_wavelet-LLL_gldm_SmallDependenceLowGrayLevelEmphasis            1
Length: 2924, dtype: int64
Total missing values in df_1: 12557
NHT组学数据整体数量： 766080
Percentage of missing values in NHT: 1.64%


In [14]:
######################################  PTMC null check and statistics ##################################################
# check all null
# df_1_Nan=df_1.isnull().sum()
# df_2_Nan=df_2.isnull().sum()
# print('df_1_NHT缺失值：',"\n", df_1_Nan,"\n")
# print('df_2_PTMC缺失值：',"\n", df_2_Nan,"\n")

# display columns of null and show total null number
PTMC_missing_data = df_2.isnull().sum()
PTMC_missing_data = PTMC_missing_data[PTMC_missing_data > 0].sort_values(ascending=False)
total_missing_values = PTMC_missing_data.sum()
print("PTMC Columns with missing values:")
print(PTMC_missing_data)
print(f"Total missing values in df_1: {total_missing_values}")

# show number of PTMC DataFrame and Percentage of missing values in PTMC
total_data_points = df_2.size 
print("PTMC组学数据整体数量：", total_data_points) 
missing_percentage = (total_missing_values / total_data_points) * 100
print(f"Percentage of missing values in PTMC: {missing_percentage:.2f}%")

PTMC Columns with missing values:
T1_C_original_shape_Elongation                          5
T1_C_wavelet-LLH_glrlm_LongRunEmphasis                  5
T1_C_wavelet-LLH_glszm_GrayLevelNonUniformity           5
T1_C_wavelet-LLH_glrlm_ShortRunLowGrayLevelEmphasis     5
T1_C_wavelet-LLH_glrlm_ShortRunHighGrayLevelEmphasis    5
                                                       ..
T1_wavelet-HLH_glcm_ClusterProminence                   3
T1_wavelet-HLH_glcm_JointAverage                        3
T1_wavelet-HLH_glcm_Autocorrelation                     3
T1_wavelet-HLH_firstorder_Variance                      3
T1_original_shape_Elongation                            3
Length: 1920, dtype: int64
Total missing values in df_1: 7680
PTMC组学数据整体数量： 510720
Percentage of missing values in PTMC: 1.50%


In [15]:

# Missing Values using Imputer （Simple Imputer）
imp = SimpleImputer(missing_values=np.NaN, strategy='mean')
imp = imp.fit(df_1)

df_1=imp.transform(df_1)
df_2=imp.transform(df_2)
df_1=pd.DataFrame(data=df_1, columns=df_1_colNames)
df_2=pd.DataFrame(data=df_2, columns=df_2_colNames)

df_1_Nan_imp=df_1.isnull().sum()
df_2_Nan_imp=df_2.isnull().sum()
print('df_1缺失值处理后：',"\n", df_1_Nan_imp,"\n") 
print('df_2缺失值处理后：',"\n", df_2_Nan_imp,"\n") 

df_1.to_excel(writer_1, sheet_name='NHT_imputer2', index=False)
writer_1.save()
writer_1.close()
df_2.to_excel(writer_2, sheet_name='PTMC_imputer2', index=False)
writer_2.save()
writer_2.close()

df_1缺失值处理后： 
 ADC_original_shape_Elongation                               0
ADC_original_shape_Flatness                                 0
ADC_original_shape_LeastAxisLength                          0
ADC_original_shape_MajorAxisLength                          0
ADC_original_shape_Maximum2DDiameterColumn                  0
                                                           ..
T2_wavelet-LLL_gldm_LargeDependenceLowGrayLevelEmphasis     0
T2_wavelet-LLL_gldm_LowGrayLevelEmphasis                    0
T2_wavelet-LLL_gldm_SmallDependenceEmphasis                 0
T2_wavelet-LLL_gldm_SmallDependenceHighGrayLevelEmphasis    0
T2_wavelet-LLL_gldm_SmallDependenceLowGrayLevelEmphasis     0
Length: 6720, dtype: int64 

df_2缺失值处理后： 
 ADC_original_shape_Elongation                               0
ADC_original_shape_Flatness                                 0
ADC_original_shape_LeastAxisLength                          0
ADC_original_shape_MajorAxisLength                          0
ADC_original_